In [3]:
import os
import time
import json
import pandas as pd
from openai import OpenAI
from transformers import AutoTokenizer
from get_params import DATA_FOLDER, SHARED_DATA_FOLDER
# from get_params import INPUT_DATASET_INFERENCE_FPATH, INPUT_DATASET_EVAL_FPATH
from get_params import TEST_DATASET_INFERENCE_FPATH, TEST_DATASET_EVAL_FPATH

# ## What data to run on
# DATASET_INFERENCE_FPATH = INPUT_DATASET_INFERENCE_FPATH
# DATASET_EVAL_FPATH = INPUT_DATASET_EVAL_FPATH
# SYSTEM_A_COL = "standard_output"
# SYSTEM_B_COL = "empathy_output"

# DATASET_INFERENCE_FPATH = TEST_DATASET_INFERENCE_FPATH
# DATASET_EVAL_FPATH = TEST_DATASET_EVAL_FPATH
# SYSTEM_A_COL = "base_output"
# SYSTEM_B_COL = "dpo_output"

DATASET_INFERENCE_FPATH = os.path.join(SHARED_DATA_FOLDER, "dpo_dataset_test_output_base_dpo_v1.jsonl")
DATASET_EVAL_FPATH = os.path.join(SHARED_DATA_FOLDER, "dpo_dataset_test_gpt_as_judge_output_v1_new_metrics.jsonl")
SYSTEM_A_COL = "base_output"
SYSTEM_B_COL = "dpo_output"


## Get OpenAI Key
with open(os.path.join(DATA_FOLDER, "openai_key.txt")) as f:
    OPENAI_KEY = f.read().strip()


## Get HF tokenizer
with open(os.path.join(DATA_FOLDER, "hftoken.txt")) as f:
    HF_TOKEN = f.read().strip()


MODEL_NAME = "gpt-4o"
MODEL_TEMPERATURE = 0.0



In [4]:
## Combining base and dpo output if they are separate

# bfpath = os.path.join(SHARED_DATA_FOLDER, "dpo_dataset_test_output_base.jsonl")
# dfpath = os.path.join(SHARED_DATA_FOLDER, "dpo_dataset_test_output_dpo.jsonl")

# bdf = pd.read_json(bfpath, lines=True)
# ddf = pd.read_json(dfpath, lines=True)

# display(bdf.head(2))
# display(ddf.head(2))

# bdf["dpo_output"] = ddf["dpo_output"].to_list()
# display(bdf.head(2))

# bdf.to_json(TEST_DATASET_OUTPUT_FPATH, orient="records", lines=True)


In [5]:
output_df = pd.read_json(DATASET_INFERENCE_FPATH, lines=True)

# add id col 
output_df['id'] = list(range(1, len(output_df)+1))

In [6]:
print (len(output_df))
display(output_df.head(2))

# print(output_df["title"].iloc[0])
# print(output_df["body"].iloc[0])

# print ("\nSTANDARD OUTPUT")
# print(output_df["standard_output"].iloc[100])

# print ("\nEMPATHY OUTPUT")
# print(output_df["empathy_output"].iloc[100])

display(output_df.head(2))
display(output_df["base_output"].iloc[0])
display(output_df["dpo_output"].iloc[0])


394


,title,body,topic,prompt,base_output,dpo_output,id
0,Feeling Stuck with Potty Training,My 1-year-old seems to show interest in the po...,Potty training struggles - Ages 1-2,<s>[INST] You are a parenting assistant. Your ...,It's essential to remember that every child is...,I understand your concern and it's important t...,1
1,Potty Training Pride with a Hint of Doubt,"Last week, my 1-year-old actually used the pot...",Potty training struggles - Ages 1-2,<s>[INST] You are a parenting assistant. Your ...,Congratulations on your child's first successf...,I'm glad to hear about your child's first succ...,2


,title,body,topic,prompt,base_output,dpo_output,id
0,Feeling Stuck with Potty Training,My 1-year-old seems to show interest in the po...,Potty training struggles - Ages 1-2,<s>[INST] You are a parenting assistant. Your ...,It's essential to remember that every child is...,I understand your concern and it's important t...,1
1,Potty Training Pride with a Hint of Doubt,"Last week, my 1-year-old actually used the pot...",Potty training struggles - Ages 1-2,<s>[INST] You are a parenting assistant. Your ...,Congratulations on your child's first successf...,I'm glad to hear about your child's first succ...,2


"It's essential to remember that every child is unique, and potty training can vary greatly. While your child may show interest, it doesn't necessarily mean they are ready. The American Academy of Pediatrics suggests that children may start showing signs of readiness between 18 and 24 months, but some may not be ready until they are 3 years old.\n\nSigns of readiness include staying dry for at least two hours at a time, showing interest in the potty, and being able to follow simple instructions. If your child is not showing these signs, it's okay to wait and try again later.\n\nTo make the process less stressful, try to approach potty training as a gradual process rather than a rushed one. Here are some tips:\n\n1. Start by reading books about potty training with your child.\n2. Let your child watch you use the potty and explain what you're doing.\n3. Encourage your child to sit on the potty for short periods, even if they don't go.\n4. Praise your child for trying, even if they have a

'I understand your concern and it\'s important to remember that every child is unique and develops at their own pace. Potty training often starts between 18 months and 3 years, but some children may show interest earlier or later.\n\nYour daughter might be showing interest, but the actual training might be overwhelming for her. It\'s crucial to respect her feelings and not force the issue. Instead, try to create a relaxed and positive environment around potty training.\n\nHere are some tips to make the process less stressful:\n\n1. **Observation**: Notice when she shows signs of needing to go, such as squirming or making faces. This can help you understand her body\'s signals better.\n\n2. **Involvement**: Let her help with dressing and undressing, and sit her on the potty during her usual bathroom times. This can help her associate the potty with elimination.\n\n3. **Positive Reinforcement**: Praise her when she sits on the potty, even if she doesn\'t go. Use simple, positive phrases 

In [7]:
SYSTEM_PROMPT = """You are an impartial judge. Your task is to compare two AI model responses from system A and system B. 
Both systems are acting as parenting assistants to assist users who are parents. 
Parents asked both systems the same parenting question, and received responses. 

Compare the two responses based on the following evaluation criteria:
1. Adoptability – How likely is the user to act on the advice? Responses that empathetic in tone while offering practical, accurate and clear advice are more adoptable. 
2. Empathy & Tone – Is the response empathetic, warm and supportive?
3. Clarity – Is the response easy to follow?
4. Comprehensiveness & Accuracy – Does the response cover the important aspects of the parenting situation with relevant, accurate info?
5. Practical Steps – Does the response give step-by-step practical actions parents can follow?

Output:
- For each of the 5 criteria above, pick which system 'won' (is better than the other) or if there's a tie: System A, System B, or Tie. 
- For each criteria, avoid ties whenever possible.
- Finally, choose an overall winner based on what you think a user is likely to choose as their preferred response: System A or System B.
- Brief justification for your overall winner choice. 
"""

USER_PROMPT = """User's Question Title: {qtitle}

User's Question Body: 
{qbody}

System A's Response:
{mistral_response}

System B's Response:
{pretendparentai_response}
"""


In [8]:
def get_pcts(a, t):
    return round((a/t)*100.0, 0)
    
def run_eval(test_all_df, system_a_col, system_b_col):
    client = OpenAI(api_key=OPENAI_KEY)
    cnt_a_wins = 0
    cnt_b_wins = 0
    total_evaluated = 0
    evaluations = []
    for i in range(len(test_all_df)):
        user_prompt_formatted = USER_PROMPT.format(
            qtitle = test_all_df["title"].iloc[i],
            qbody = test_all_df["body"].iloc[i],
            mistral_response = test_all_df[system_a_col].iloc[i],
            pretendparentai_response = test_all_df[system_b_col].iloc[i]
        )
        resp = client.responses.create(
            model=MODEL_NAME,
            temperature=MODEL_TEMPERATURE,
            instructions=SYSTEM_PROMPT,
            input=user_prompt_formatted,
            text={
                "format": {
                    "type": "json_schema",
                    "name": "eval",
                    "strict": True,
                    "schema": {
                        "type": "object",
                            "properties": {
                                ## Comparison
                                "winner_adoptability": {
                                    "type": "string",
                                    "enum": ["System A", "System B", "Tie"],
                                    "description": "Which response is the user most likely to adopt and act on?"
                                },
                                "winner_empathy": {
                                    "type": "string",
                                    "enum": ["System A", "System B", "Tie"],
                                    "description": "Which system's response is more empathetic, warm, and supportive?"
                                },
                                "winner_clarity": {
                                    "type": "string",
                                    "enum": ["System A", "System B", "Tie"],
                                    "description": "Which system's response is easier to follow?"
                                },
                                "winner_comprehensiveness": {
                                    "type": "string",
                                    "enum": ["System A", "System B", "Tie"],
                                    "description": "Which system's response is more comprehensive and accurate?"
                                },
                                "winner_practical": {
                                    "type": "string",
                                    "enum": ["System A", "System B", "Tie"],
                                    "description": "Which system's response offers more step-by-step practical actions for parents to follow?"
                                },
                                "winner_overall": {
                                    "type": "string",
                                    "enum": ["System A", "System B"],
                                    "description": "Which system's response would users prefer?"
                                },
                                "justification": {
                                    "type": "string",
                                    "description": "Justify your reasoning for your overall winner choice."
                                },
                            },
                        "required": [
                            "winner_adoptability",
                            "winner_empathy",
                            "winner_clarity",
                            "winner_comprehensiveness",
                            "winner_practical",
                            "winner_overall",
                            "justification"
                        ],
                        "additionalProperties": False
                    }
                }
            }
        )
        parsed = json.loads(resp.output_text)
        parsed["id"] = test_all_df["id"].iloc[i]
        total_evaluated += 1
        if parsed['winner_overall'].lower() == "system a":
            cnt_a_wins += 1
        elif parsed['winner_overall'].lower() == "system b":
            cnt_b_wins += 1
        print (f"{get_pcts(cnt_a_wins+cnt_b_wins, total_evaluated)} are not adoptability ties")
        print (f"{parsed['id']} {parsed['winner_overall']} (Em: {parsed['winner_empathy']}, Ad: {parsed['winner_adoptability']}) (A: {get_pcts(cnt_a_wins, total_evaluated)}, B: {get_pcts(cnt_b_wins, total_evaluated)}): {parsed['justification']} ")
        evaluations.append(parsed)
    return evaluations


In [9]:
evaluations = run_eval(output_df, SYSTEM_A_COL, SYSTEM_B_COL)


100.0 are not adoptability ties
1 System B (Em: System B, Ad: System B) (A: 0.0, B: 100.0): System B's response is more empathetic and supportive, addressing the parent's feelings and emphasizing patience and understanding. It offers practical steps that are easy to implement, such as observing the child's signals and encouraging independence. While System A provides comprehensive information, System B's tone and practical advice make it more likely for the user to adopt and act on the suggestions, making it the preferred choice overall. 
100.0 are not adoptability ties
2 System A (Em: System A, Ad: System A) (A: 50.0, B: 50.0): System A's response is more adoptable due to its clear, structured, and practical advice. It provides a comprehensive list of actionable steps that are easy to follow, such as using fun aids and modeling behavior. The tone is empathetic and supportive, acknowledging the parent's feelings and offering encouragement. System B also offers good advice but lacks the

In [10]:
eval_df = pd.DataFrame(evaluations)


In [11]:
# Merge with output_df to get eval_output_df
eval_output_df = output_df.merge(eval_df, how="inner", on="id")
print (len(eval_output_df))
eval_output_df.head(2)

394


,title,body,topic,prompt,base_output,dpo_output,id,winner_adoptability,winner_empathy,winner_clarity,winner_comprehensiveness,winner_practical,winner_overall,justification
0,Feeling Stuck with Potty Training,My 1-year-old seems to show interest in the po...,Potty training struggles - Ages 1-2,<s>[INST] You are a parenting assistant. Your ...,It's essential to remember that every child is...,I understand your concern and it's important t...,1,System B,System B,System A,System A,System B,System B,System B's response is more empathetic and sup...
1,Potty Training Pride with a Hint of Doubt,"Last week, my 1-year-old actually used the pot...",Potty training struggles - Ages 1-2,<s>[INST] You are a parenting assistant. Your ...,Congratulations on your child's first successf...,I'm glad to hear about your child's first succ...,2,System A,System A,System A,System A,System A,System A,System A's response is more adoptable due to i...


In [12]:
eval_output_df.to_csv(DATASET_EVAL_FPATH)


In [13]:
norm_counts = pd.concat(
    {col: eval_df[col].value_counts(normalize=True) 
     for col in eval_df.columns if col not in ["justification", "id"]},
    axis=1
).fillna(0)



In [14]:
norm_counts

,winner_adoptability,winner_empathy,winner_clarity,winner_comprehensiveness,winner_practical,winner_overall
System A,0.725888,0.106599,0.888325,0.708122,0.840102,0.746193
System B,0.274112,0.763959,0.111675,0.276650,0.157360,0.253807
Tie,0.000000,0.129442,0.000000,0.015228,0.002538,0.000000


In [15]:
features = [x for x in eval_df.columns if x not in ["id", "winner_overall", "justification"]]
features_out = {}
idx = ["System A", "System B"]

for f in features:
    features_out[f] = len(eval_df[(eval_df[f]==eval_df["winner_overall"])])

features_out

{'winner_adoptability': 386,
 'winner_empathy': 127,
 'winner_clarity': 338,
 'winner_comprehensiveness': 346,
 'winner_practical': 351}